In [1]:
import os 

In [2]:
%pwd

'c:\\Projects\\Kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Radhwen-HAJRI/Kidney-disease-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Radhwen-HAJRI"
os.environ["MLFLOW_TRACKING_PASSWORD"]="78e5e078c134d008091b62c2d4e5c913bba61b00"

In [5]:
import dagshub
dagshub.init(repo_owner='Radhwen-HAJRI', repo_name='Kidney-disease-classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Radhwen-HAJRI

Initialized MLflow to track repo "Radhwen-HAJRI/Kidney-disease-classification"

Repository Radhwen-HAJRI/Kidney-disease-classification initialized!

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model(r"C:\Projects\Kidney-disease-classification\artifacts\training\trained_model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int
     

In [9]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=config_file_path,
                 params_filepath=params_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=r"artifacts\training\trained_model.h5",
            training_data=r"artifacts\data_ingestion\kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Radhwen-HAJRI/Kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [12]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale= 1./255,
            validation_split = 0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self._valid_generator=valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle=False,
            class_mode="sparse",
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    def Evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self._valid_generator)
        
    def save_score(self):
        score = {
            "loss":self.score[0],
            "accuracy":self.score[1]
        }
        save_json(path=Path("scores.json"),data=score)
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        training_url_type_store = urlparse(self.config.mlflow_uri).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("loss",self.score[0])
            mlflow.log_metric("accuracy",self.score[1])
            
            if training_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model", registered_model_name="VGG16_Kidney_disease_classification")
            else:
                mlflow.keras.log_model(self.model,"model")
                

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.Evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 2207 images belonging to 2 classes.
138/138 [==============================] - 125s 902ms/step - loss: 1.6123 - accuracy: 0.6914


2025/10/27 11:27:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Hajri\AppData\Local\Temp\tmp7zaei34g\model\data\model\assets


c:\Users\Hajri\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16_Kidney_disease_classification' already exists. Creating a new version of this model...
2025/10/27 11:28:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16_Kidney_disease_classification, version 3
Created version '3' of model 'VGG16_Kidney_disease_classification'.
